<a href="https://colab.research.google.com/github/rishubhkhurana/nlp/blob/main/attention/Model2_BreakData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libs

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import re,os,sys,gc
import json
import torchtext
from torchtext.data import Field, Example, BucketIterator
from torchtext import data
import spacy, random
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import time, math

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Download data

In [3]:
!wget https://github.com/allenai/Break/raw/master/break_dataset/Break-dataset.zip

--2020-12-29 12:39:09--  https://github.com/allenai/Break/raw/master/break_dataset/Break-dataset.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/allenai/Break/master/break_dataset/Break-dataset.zip [following]
--2020-12-29 12:39:09--  https://raw.githubusercontent.com/allenai/Break/master/break_dataset/Break-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15971078 (15M) [application/zip]
Saving to: ‘Break-dataset.zip’

Break-dataset.zip   100%[===================>]  15.23M  94.2MB/s    in 0.2s    

2020-12-29 12:39:10 (94.2 MB/s) - ‘Break-dataset.zip’ saved [15971078/15971078]



In [4]:
!unzip Break-dataset.zip

Archive:  Break-dataset.zip
   creating: Break-dataset/logical-forms/
  inflating: Break-dataset/logical-forms/dev.csv  
  inflating: Break-dataset/logical-forms/test.csv  
  inflating: Break-dataset/logical-forms/train.csv  
   creating: Break-dataset/QDMR/
  inflating: Break-dataset/QDMR/dev.csv  
  inflating: Break-dataset/QDMR/dev_lexicon_tokens.json  
  inflating: Break-dataset/QDMR/test.csv  
  inflating: Break-dataset/QDMR/test_lexicon_tokens.json  
  inflating: Break-dataset/QDMR/train.csv  
  inflating: Break-dataset/QDMR/train_lexicon_tokens.json  
   creating: Break-dataset/QDMR-high-level/
  inflating: Break-dataset/QDMR-high-level/dev.csv  
  inflating: Break-dataset/QDMR-high-level/dev_lexicon_tokens.json  
  inflating: Break-dataset/QDMR-high-level/test.csv  
  inflating: Break-dataset/QDMR-high-level/test_lexicon_tokens.json  
  inflating: Break-dataset/QDMR-high-level/train.csv  
  inflating: Break-dataset/QDMR-high-level/train_lexicon_tokens.json  


## Creating Data

In [5]:
train_df = pd.read_csv('Break-dataset/QDMR/train.csv')
valid_df = pd.read_csv('Break-dataset/QDMR/dev.csv')

In [6]:
nlp = spacy.load('en')

In [7]:
def tokenize_en(text):
    return  [tok.text for tok in nlp.tokenizer(text)]

In [8]:
SRC = Field(tokenize=tokenize_en, lower=True, init_token='<sos>', eos_token='<eos>', batch_first=False)
TRG = Field(tokenize=tokenize_en, lower=True, init_token='<sos>', eos_token='<eos>', batch_first=False)

In [9]:
fields = [('src',SRC),('trg',TRG)]

In [10]:

train_examples = [Example.fromlist([s,t],fields) for s,t in zip(train_df['question_text'],train_df['decomposition'])]
valid_examples = [Example.fromlist([s,t],fields) for s,t in zip(valid_df['question_text'],valid_df['decomposition'])]

In [11]:
train_ds = data.Dataset(train_examples, fields)
valid_ds = data.Dataset(train_examples, fields)

In [12]:
SRC.build_vocab(train_ds, min_freq=2)
TRG.build_vocab(train_ds, min_freq=2)

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 128
train_iterator, valid_iterator = BucketIterator.splits(
    (train_ds, valid_ds), 
    batch_size = BATCH_SIZE, 
    device = device,sort_key = lambda x: x.src)

## Model

In [14]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
  
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [15]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [16]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [17]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

# Training Model

In [19]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [20]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(9494, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(9499, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=9499, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 28,327,195 trainable parameters


In [22]:
optimizer = optim.Adam(model.parameters())

In [23]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [24]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [25]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [26]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [27]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 11m 18s
	Train Loss: 3.047 | Train PPL:  21.050
	 Val. Loss: 4.187 |  Val. PPL:  65.857
Epoch: 02 | Time: 11m 18s
	Train Loss: 2.117 | Train PPL:   8.310
	 Val. Loss: 4.172 |  Val. PPL:  64.825
Epoch: 03 | Time: 11m 15s
	Train Loss: 1.785 | Train PPL:   5.961
	 Val. Loss: 4.026 |  Val. PPL:  56.040
Epoch: 04 | Time: 11m 15s
	Train Loss: 1.526 | Train PPL:   4.601
	 Val. Loss: 3.940 |  Val. PPL:  51.420
Epoch: 05 | Time: 11m 19s
	Train Loss: 1.336 | Train PPL:   3.803
	 Val. Loss: 3.715 |  Val. PPL:  41.076
Epoch: 06 | Time: 11m 17s
	Train Loss: 1.164 | Train PPL:   3.203
	 Val. Loss: 3.620 |  Val. PPL:  37.339
Epoch: 07 | Time: 11m 15s
	Train Loss: 1.048 | Train PPL:   2.851
	 Val. Loss: 3.523 |  Val. PPL:  33.890
Epoch: 08 | Time: 11m 21s
	Train Loss: 0.966 | Train PPL:   2.626
	 Val. Loss: 3.467 |  Val. PPL:  32.040
Epoch: 09 | Time: 11m 17s
	Train Loss: 0.885 | Train PPL:   2.422
	 Val. Loss: 3.277 |  Val. PPL:  26.502
Epoch: 10 | Time: 11m 17s
	Train Loss: 0.829 |